# Google Analytics 4 (GA4) Data in Pyton Using run_report 

In [1]:
import numpy as np
import pandas as pd
import os
from datetime import date, timedelta


In [2]:
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import DateRange
from google.analytics.data_v1beta.types import Dimension
from google.analytics.data_v1beta.types import Metric
from google.analytics.data_v1beta.types import Filter
from google.analytics.data_v1beta.types import FilterExpression
from google.analytics.data_v1beta.types import FilterExpressionList
from google.analytics.data_v1beta.types import RunReportRequest
from google.analytics.data_v1beta.types import OrderBy

In [3]:
## Set up global variables

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'bubbly-cascade-398303-75988e28019e.json'
property_id = '237200408'

client = BetaAnalyticsDataClient()

In [4]:
## Format Report - run_report method
def format_report(request):
    response = client.run_report(request)
    print(response.property_quota)
    # Row index
    row_index_names = [header.name for header in response.dimension_headers]
    row_header = []
    for i in range(len(row_index_names)):
        row_header.append([row.dimension_values[i].value for row in response.rows])

    row_index_named = pd.MultiIndex.from_arrays(np.array(row_header), names = np.array(row_index_names))
    # Row flat data
    metric_names = [header.name for header in response.metric_headers]
    data_values = []
    for i in range(len(metric_names)):
        data_values.append([row.metric_values[i].value for row in response.rows])

    output = pd.DataFrame(data = np.transpose(np.array(data_values, dtype = 'f')), 
                          index = row_index_named, columns = metric_names)
    return output

In [24]:
def format_report1(request):
    """
    Fetches GA4 data, creates a pandas DataFrame, and potentially returns it as a response.
    
    Args:
      request (Any): HTTP request object (unused in this implementation).
    
    Returns:
      pandas.DataFrame: The constructed DataFrame with GA4 data (optional).
    """
    
    response = client.run_report(request)
    
    # Row index names
    row_index_names = [header.name for header in response.dimension_headers]
    
    # Row header (list of lists)
    row_header = []
    for i in range(len(row_index_names)):
        row_header.append([row.dimension_values[i].value for row in response.rows])
    
    # Create MultiIndex for rows
    row_index_named = pd.MultiIndex.from_tuples(zip(*row_header), names=row_index_names)
    
    # Extract metric names and values
    metric_names = [header.name for header in response.metric_headers]
    data_values = [[row.metric_values[i].value for row in response.rows] for i in range(len(metric_names))]
    
    # Create DataFrame with efficient transposition
    output = pd.DataFrame(
    data=data_values[0],  # Transposition happens automatically
    index=row_index_named,
    columns=metric_names
    
    )
    
    
    # Consider potential return based on your Cloud Function's purpose
    # If you want to return the DataFrame as the response:
    # return output
    
    # If you have further processing or returning something different:
    # ... (your logic here)
    # return some_other_data
    
    return output  # Placeholder for flexible return handling


In [6]:
# Table 1
start_date=date.today() - timedelta(days=2)

date_range = DateRange(start_date=start_date.strftime("%Y-%m-%d"), end_date=start_date.strftime("%Y-%m-%d"))

request = RunReportRequest(
        property='properties/'+property_id,
        dimensions=[Dimension(name="customEvent:pnr"),Dimension(name="customEvent:storage_source"),
                    Dimension(name="customEvent:full_journey"),Dimension(name="Date")],
        metrics=[Metric(name="totalRevenue")],
        order_bys = [OrderBy(dimension = {'dimension_name': 'Date'})],
        date_ranges=[date_range],
        dimension_filter=FilterExpression(
                and_group=FilterExpressionList(
                    expressions=[
                        FilterExpression(
                            filter=Filter(
                                field_name="customEvent:storage_source",
                                string_filter=Filter.StringFilter(match_type="PARTIAL_REGEXP", value="skyscanner|Skyscanner|KAYAK"),
                            ),
                        )
                    ]
                )
        ),
        return_property_quota = True
    )

request

property: "properties/237200408"
dimensions {
  name: "customEvent:pnr"
}
dimensions {
  name: "customEvent:storage_source"
}
dimensions {
  name: "customEvent:full_journey"
}
dimensions {
  name: "Date"
}
metrics {
  name: "totalRevenue"
}
date_ranges {
  start_date: "2024-07-07"
  end_date: "2024-07-07"
}
dimension_filter {
  and_group {
    expressions {
      filter {
        field_name: "customEvent:storage_source"
        string_filter {
          match_type: PARTIAL_REGEXP
          value: "skyscanner|Skyscanner|KAYAK"
        }
      }
    }
  }
}
order_bys {
  dimension {
    dimension_name: "Date"
  }
}
return_property_quota: true

In [7]:
format_report1(request)

NameError: name 'format_report1' is not defined

In [8]:
output_df = format_report(request)

# ## Export to Excel
# output_df.reset_index().to_excel('GA4_python_output.xlsx', sheet_name = 'GA4_report', engine = 'xlsxwriter')

tokens_per_day {
  consumed: 1
  remaining: 1923509
}
tokens_per_hour {
  consumed: 1
  remaining: 252821
}
concurrent_requests {
  consumed: 0
  remaining: 50
}
server_errors_per_project_per_hour {
  consumed: 0
  remaining: 50
}
potentially_thresholded_requests_per_hour {
  consumed: 0
  remaining: 120
}
tokens_per_project_per_hour {
  consumed: 1
  remaining: 139989
}



In [9]:
output_df

,,,,totalRevenue
customEvent:pnr,customEvent:storage_source,customEvent:full_journey,Date,
5M2UV4,KAYAK,MEL-FRA:FRA-MEL,20240707,1.305394e+08
5OWKPC,skyscanner,MUC-DAD:DAD-MUC,20240707,1.190332e+08
5NSZPX,skyscanner,HAN-MUC:MUC-HAN,20240707,1.056922e+08
5Q36Z6,skyscanner,SEL-NHA:NHA-SEL,20240707,1.007506e+08
5Q5H32,skyscanner,LON-HAN:HAN-LON,20240707,9.633887e+07
...,...,...,...,...
5N74V3,skyscanner,HAN-DAD,20240707,1.222053e+06
5NGFEQ,skyscanner,HAN-DAD,20240707,1.216000e+06
5NYB5B,skyscanner,SGN-NHA,20240707,1.207911e+06


In [25]:
output = format_report1(request)

In [42]:
out = output.reset_index()

In [52]:
out = out.rename(columns={('', '',''):('pnr','storage_source','full_journey')})

In [60]:
out = out.rename(columns={'customEvent:pnr': 'pnr', 'customEvent:storage_source': 'storage_source','customEvent:full_journey':'full_journey'})

In [61]:
out.columns

Index(['pnr', 'storage_source', 'full_journey', 'Date', 'totalRevenue'], dtype='object')

In [57]:
 out.rename(columns={('customEvent:pnr', 'customEvent:storage_source','customEvent:full_journey'):('pnr','storage_source','full_journey')}, inplace = True)

In [58]:
!python -V 

Python 3.12.0


In [40]:
## Export to CSV
output_df.to_csv('GA4_SKY.csv')